In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import keras

from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization, Input
from keras.models import Sequential
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam,RMSprop,SGD

Using TensorFlow backend.


In [2]:
df = pd.read_csv("https://storage.googleapis.com/kaggle-datasets/10449/44567/monkey_labels.txt?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1545224102&Signature=n1Ndbik5gpFHTb96LiGlvvmgDhI4MFZqlZJ775bmVQ7XDpRDxINVAqBpd7UuvPL8CtpO9P00eVB9l8mk%2FQ7eqc%2Fx9RfMPJaECR4wQrGv63g3iiV%2BIad794pS2ODyyvk2rsf1bt9R9%2FVqmj47nkM29yhC9eZyYJVC%2FW02dPCzYA3x1NmzJJw8m%2BV%2FiSPmsD6572lKCSWhABXtseu5knF42TVvQIQ6a7eAn7o9XJkMxh%2BK1wqOs6jZ%2FJg3vqUAR22MPqie2y9bYTh4DXZHiGk6IxE94X85syLeEv5wCBB34n2pR6d9aOIJPOTMUkVxr92IbU6qJeGbD8lTJrxyP51rrg%3D%3D")

In [3]:
df

,Label,Latin Name,Common Name,Train Images,Validation Images
0,n0,alouatta_palliata\t,mantled_howler,131,26
1,n1,erythrocebus_patas\t,patas_monkey,139,28
2,n2,cacajao_calvus\t,bald_uakari,137,27
3,n3,macaca_fuscata\t,japanese_macaque,152,30
4,n4,cebuella_pygmea\t,pygmy_marmoset,131,26
5,n5,cebus_capucinus\t,white_headed_capuchin,141,28
6,n6,mico_argentatus\t,silvery_marmoset,132,26
7,n7,saimiri_sciureus\t,common_squirrel_monkey,142,28
8,n8,aotus_nigriceps\t,black_headed_night_monkey,133,27
9,n9,trachypithecus_johnii,nilgiri_langur,132,26


In [4]:
train_datagen = ImageDataGenerator(rotation_range = 30
                                   ,rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

In [6]:
height=150
width=150
channels=3
batch_size=32
seed=1337


train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory("/Users/yhaddad/Downloads/training/", 
                                                  target_size=(height,width), 
                                                  batch_size=batch_size,
                                                  seed=seed,
                                                  class_mode='categorical')

Found 1097 images belonging to 10 classes.


In [7]:
from keras.applications.vgg16 import VGG16
conv_base = VGG16(include_top= False, input_shape=(150,150,3))
conv_base.summary()

58892288/58889256 [==============================] - 6s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool

In [8]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(Adam(lr=0.0001),loss="categorical_crossentropy", metrics=["accuracy"])
#model.fit(train_x, train_y, batch_size=32, validation_data = (test_x, test_y), epochs=10)
history = model.fit_generator(train_generator,
                    steps_per_epoch = 1097//batch_size,
                    validation_data = test_generator,
                    validation_steps = 4, epochs = 10)

Epoch 1/10
19/34 [===============>..............] - ETA: 6:51 - loss: 2.3615 - acc: 0.0921